<a href="https://colab.research.google.com/github/david-meltzer/Goodreads-Sentiment-Analysis/blob/main/QG_Consistent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies

In [1]:
!pip install transformers[sentencepiece] datasets evaluate
!pip install wandb

# Mount google drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
#foldername should be changed to the name of the directory containing the file.
foldername = '/Question_Generation/Longform-Question-Generation/'
%cd /content/drive/My\ Drive/$foldername

import wandb, transformers, torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
import os

device = "cuda" if torch.cuda.is_available() else "cpu"

sys.path.append('/content/drive/My Drive/{}'.format(foldername))    

#creates the 'results' and 'data' directories, if they don't exist.
if not os.path.exists('results'):
    os.makedirs('results')

if not os.path.exists('data'):
    os.makedirs('data')

from datasets import load_from_disk, load_dataset, Dataset, load_metric
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    BartForConditionalGeneration,
    BartTokenizer
)

from torch.utils.data import Dataset, DataLoader

import evaluate
from transformers.trainer_callback import (EarlyStoppingCallback, 
                                           TrainerCallback)

from huggingface_hub import notebook_login
notebook_login()

!git pull

Already up to date.


# ELI5

In [ ]:
dataset = load_dataset('eli5')

Downloading:   0%|          | 0.00/3.50k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/576M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/21.1M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/53.0M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/286M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/330M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/18.7M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/36.2M [00:00<?, ?B/s]

Dataset eli5 downloaded and prepared to /root/.cache/huggingface/datasets/eli5/LFQA_reddit/1.0.0/17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa. Subsequent calls will reuse this data.


  0%|          | 0/9 [00:00<?, ?it/s]

In [ ]:
dataset.set_format('pandas')

In [ ]:
dataset['train_asks'].features

{'q_id': Value(dtype='string', id=None),
 'title': Value(dtype='string', id=None),
 'selftext': Value(dtype='string', id=None),
 'document': Value(dtype='string', id=None),
 'subreddit': Value(dtype='string', id=None),
 'answers': Sequence(feature={'a_id': Value(dtype='string', id=None), 'text': Value(dtype='string', id=None), 'score': Value(dtype='int32', id=None)}, length=-1, id=None),
 'title_urls': Sequence(feature={'url': Value(dtype='string', id=None)}, length=-1, id=None),
 'selftext_urls': Sequence(feature={'url': Value(dtype='string', id=None)}, length=-1, id=None),
 'answers_urls': Sequence(feature={'url': Value(dtype='string', id=None)}, length=-1, id=None)}

In [ ]:
dataset['train_askh']

Dataset({
    features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'answers', 'title_urls', 'selftext_urls', 'answers_urls'],
    num_rows: 98525
})

# Bart-Base

In [ ]:
# Initialize BART tokenizer and model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

#Scratch

In [ ]:
# Define custom dataset class
class CustomDataset(Dataset):
    def __init__(self, examples):
        self.examples = examples

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        return self.examples[idx]

# Example training data
train_data = [
    {'input': 'I am hungry', 'target': 'What should I eat?'},
    {'input': 'The weather is nice today', 'target': 'What activities can I do outside?'},
    {'input': 'I have a headache', 'target': 'What can I take for it?'},
]

# Initialize BART tokenizer and model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

# Prepare data for fine-tuning
train_dataset = CustomDataset(train_data)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

# Fine-tune BART on the conditional generation task
for epoch in range(3):
    for batch in train_loader:
        input_batch = tokenizer(batch['input'], padding=True, truncation=True, return_tensors='pt')
        target_batch = tokenizer(batch['target'], padding=True, truncation=True, return_tensors='pt')

        input_ids = input_batch['input_ids']
        attention_mask = input_batch['attention_mask']
        target_ids = target_batch['input_ids']
        target_attention_mask = target_batch['attention_mask']

        loss = model(input_ids=input_ids, attention_mask=attention_mask, decoder_input_ids=target_ids[:, :-1], 
                     decoder_attention_mask=target_attention_mask[:, :-1], labels=target_ids[:, 1:]).loss

        loss.backward()
        optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
        optimizer.step()
        optimizer.zero_grad()

        print('Epoch:', epoch, 'Batch Loss:', loss.item())


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("bigscience/T0pp")
model = AutoModelForSeq2SeqLM.from_pretrained("bigscience/T0pp")

inputs = tokenizer.encode("Is this review positive or negative? Review: this is the best cast iron skillet you will ever buy", return_tensors="pt")
outputs = model.generate(inputs)
print(tokenizer.decode(outputs[0]))


In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration
from torch.utils.data import Dataset, DataLoader

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("bigscience/T0_3B")
model = AutoModelForSeq2SeqLM.from_pretrained("bigscience/T0_3B")

inputs = tokenizer.encode("Is this review positive or negative? Review: this is the best cast iron skillet you will ever buy", return_tensors="pt")
outputs = model.generate(inputs)
print(tokenizer.decode(outputs[0]))	

In [ ]:
model

NameError: ignored

In [ ]:
from huggingface_hub import InferenceApi

inference = InferenceApi("bigscience/bloom",token=HfFolder.get_token())

In [ ]:
import time

def infer(prompt,
          max_length = 32,
          top_k = 0,
          num_beams = 0,
          no_repeat_ngram_size = 2,
          top_p = 0.9,
          seed=42,
          temperature=0.7,
          greedy_decoding = False,
          return_full_text = False):
    

    top_k = None if top_k == 0 else top_k
    do_sample = False if num_beams > 0 else not greedy_decoding
    num_beams = None if (greedy_decoding or num_beams == 0) else num_beams
    no_repeat_ngram_size = None if num_beams is None else no_repeat_ngram_size
    top_p = None if num_beams else top_p
    early_stopping = None if num_beams is None else num_beams > 0

    params = {
        "max_new_tokens": max_length,
        "top_k": top_k,
        "top_p": top_p,
        "temperature": temperature,
        "do_sample": do_sample,
        "seed": seed,
        "early_stopping":early_stopping,
        "no_repeat_ngram_size":no_repeat_ngram_size,
        "num_beams":num_beams,
        "return_full_text":return_full_text
    }
    
    s = time.time()
    response = inference(prompt, params=params)
    #print(response)
    proc_time = time.time()-s
    #print(f"Processing time was {proc_time} seconds")
    return response

In [ ]:
prompt = "The thing that makes large language models interesting is"
resp = infer(prompt)

resp

In [ ]:
prompt = 'TL;dr A recent study suggests the number of people making less than $1.90 a day in Myanmar has more than tripled, to 63 percent of the population, since the pandemic began. Now, after a year of Covid-19-induced shutdowns, layoffs, pay cuts and union crackdowns, tens of thousands of garment workers have been spurred into civil disobedience in recent weeks, as the coup threatens both their freedom and the industry.'

In [ ]:
resp = infer(prompt)

resp

[{'generated_text': ' On February 1, Myanmar’s military staged a coup and arrested de facto leader Aung San Suu Kyi and other members of her ruling National League for Democracy party.'}]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import os
import json

tokenizer = AutoTokenizer.from_pretrained("bigscience/T0_3B")
model = AutoModelForSeq2SeqLM.from_pretrained("bigscience/T0_3B")
model.eval()

In [9]:
prompt = 'Generate a question: A recent study suggests the number of people making less than $1.90 a day in Myanmar has more than tripled, to 63 percent of the population, since the pandemic began. Now, after a year of Covid-19-induced shutdowns, layoffs, pay cuts and union crackdowns, tens of thousands of garment workers have been spurred into civil disobedience in recent weeks, as the coup threatens both their freedom and the industry.'

In [24]:
inputs = tokenizer.encode(prompt, return_tensors="pt")
outputs = model.generate(inputs)
tokenizer.decode(outputs[0])
#print(tokenizer.decode(outputs[0]))

'<pad> What is the average daily wage in Myanmar?</s>'

In [13]:
from huggingface_hub import notebook_login
from huggingface_hub import HfFolder

from huggingface_hub import InferenceApi

inference = InferenceApi("bigscience/bloom",token=HfFolder.get_token())

import time

def infer(prompt,
          max_length = 32,
          top_k = 0,
          num_beams = 0,
          no_repeat_ngram_size = 2,
          top_p = 0.9,
          seed=42,
          temperature=0.7,
          greedy_decoding = False,
          return_full_text = False):
    

    top_k = None if top_k == 0 else top_k
    do_sample = False if num_beams > 0 else not greedy_decoding
    num_beams = None if (greedy_decoding or num_beams == 0) else num_beams
    no_repeat_ngram_size = None if num_beams is None else no_repeat_ngram_size
    top_p = None if num_beams else top_p
    early_stopping = None if num_beams is None else num_beams > 0

    params = {
        "max_new_tokens": max_length,
        "top_k": top_k,
        "top_p": top_p,
        "temperature": temperature,
        "do_sample": do_sample,
        "seed": seed,
        "early_stopping":early_stopping,
        "no_repeat_ngram_size":no_repeat_ngram_size,
        "num_beams":num_beams,
        "return_full_text":return_full_text
    }
    
    s = time.time()
    response = inference(prompt, params=params)
    #print(response)
    proc_time = time.time()-s
    #print(f"Processing time was {proc_time} seconds")
    return response


In [23]:
prompt = 'A recent study suggests the number of people making less than $1.90 a day in Myanmar has \
more than tripled, to 63 percent of the population, since the pandemic began.\
 Now, after a year of Covid-19-induced shutdowns, layoffs, pay cuts and union crackdowns,\
 tens of thousands of garment workers have been spurred into civil disobedience in recent weeks, \
 as the coup threatens both their freedom and the industry. Generate a single question'

resp = infer(prompt)

resp

[{'generated_text': ' on a single slide. This is the fourth coup in Myanmar’s history since independence in 1948. What’s the story of the Myanmar coup? The Myanmar military has'}]